In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import numpy as np
import cv2
from keras.optimizers import Adam
from keras.initializers import RandomNormal
from numpy.random import randint
from keras.models import Model
from keras.models import Input
from keras.layers import Dense
from keras.utils import np_utils
from keras.layers import Activation
from keras.layers import LeakyReLU
import tensorflow as tf
import numpy as np
import sys
import matplotlib.pyplot as plt
from numpy import argmax
from keras.preprocessing.image import ImageDataGenerator
from sklearn.preprocessing import LabelEncoder
import cv2
from tensorflow.keras.applications import DenseNet121
from sklearn.model_selection import train_test_split
from tensorflow.keras import Sequential
from tensorflow.keras import losses
import os

In [ ]:
path_cap = "/content/drive/MyDrive/IndividualProject/SPGC-COVID/data/cap/"
path_normal = "/content/drive/MyDrive/IndividualProject/SPGC-COVID/data/normal/"

In [ ]:
normal_image = []
cap_image = []
normal_files = os.listdir(path_normal)
cap_files = os.listdir(path_cap)

In [ ]:
print('/content/drive/MyDrive/IndividualProject/SPGC-COVID/data/normal/' + normal_files[0])
print(len(normal_files))
print(len(cap_files))

In [ ]:
count = 0
for file in normal_files:
  count = count + 1
  if count % 200 == 0:
    print(count)
  temp = path_normal + file
  img = cv2.imread(temp)
  img = np.array(img)
  img = (img/255).astype("float32")
  normal_image.append(img)
del normal_files
print(len(normal_image))


count = 0
for file in cap_files:
  count = count + 1
  if count % 200 == 0:
    print(count)
  temp = path_covid + file
  img = cv2.imread(temp)
  img = np.array(img)
  img = (img/255).astype("float32")
  cap_image.append(img)
del covid_files
print(len(cap_image))

In [ ]:
ix1 = randint(0, len(normal_image), len(cap_image))
print(ix1.shape)
print(ix1[0])

for i in ix1:
  count = count + 1
  if count % 100 == 0:
    print(count)
  cap_image.append(normal_image[i])
  Y1.append(0)
print(len(cap_image))
del normal_image
del ix1

In [ ]:
datagen = ImageDataGenerator(featurewise_center=True)
datagen.fit(cap)
mean = datagen.mean
cap = cap-mean
cap = np.reshape(cap,(len(cap),512,512,3))
print(cap.shape)
Y = np.array(Y)
print(mean)
mean = None

(1820, 512, 512, 3)
[[[0.03516484 0.03516484 0.03516484]]]


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(cap, Y, test_size=0.2, random_state=42)

In [ ]:
model1 = Sequential([
    DenseNet121(include_top=False, weights='imagenet', input_shape=(512,512,3)),
    tf.keras.layers.GlobalAveragePooling2D(),
    Dense(1024,activation='relu'),
    Dense(1, activation='sigmoid')
])
model1.build(input_shape=(512,512,3))
model1.summary()
model1.compile(loss='binary_crossentropy', optimizer=Adam(learning_rate=0.0001, beta_1=0.5, decay=2e-2), metrics=['accuracy'])

Model: "sequential_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
densenet121 (Functional)     (None, 16, 16, 1024)      7037504   
_________________________________________________________________
global_average_pooling2d_8 ( (None, 1024)              0         
_________________________________________________________________
module_wrapper_16 (ModuleWra (None, 1024)              1049600   
_________________________________________________________________
module_wrapper_17 (ModuleWra (None, 1)                 1025      
Total params: 8,088,129
Trainable params: 8,004,481
Non-trainable params: 83,648
_________________________________________________________________


In [ ]:
model1.fit(
    X_train,
    y_train,
    validation_data=(X_test,y_test),
    batch_size=10,
    epochs=4,
    shuffle=True,
)

Epoch 1/4
146/146 [==============================] - 989s 7s/step - loss: 0.2021 - accuracy: 0.9251 - val_loss: 0.8122 - val_accuracy: 0.6126
Epoch 2/4
146/146 [==============================] - 985s 7s/step - loss: 0.0334 - accuracy: 0.9945 - val_loss: 0.1555 - val_accuracy: 0.9313
Epoch 3/4
146/146 [==============================] - 978s 7s/step - loss: 0.0364 - accuracy: 0.9897 - val_loss: 0.0234 - val_accuracy: 1.0000
Epoch 4/4
146/146 [==============================] - 959s 7s/step - loss: 0.0139 - accuracy: 0.9979 - val_loss: 0.0066 - val_accuracy: 1.0000


In [ ]:
model1.save_weights("/content/drive/MyDrive/IndividualProject/SPGC-COVID/CAP_DenseNet/DesNet-covid-normal.ckpt")

In [ ]:
model2 = Sequential([
    DenseNet121(include_top=False, weights='imagenet', input_shape=(512,512,3)),
    tf.keras.layers.GlobalAveragePooling2D(),
    Dense(1024,activation='relu'),
    Dense(1, activation='sigmoid')
])

In [ ]:
model2.load_weights("/content/drive/MyDrive/IndividualProject/SPGC-COVID/CAP_DenseNet/DesNet-covid-normal.ckpt")

In [ ]:
img = np.array(X_train[0])
img = np.reshape(img,(1,512,512,3))

In [ ]:
print(model1(img))

tf.Tensor([[0.9987438]], shape=(1, 1), dtype=float32)


In [ ]:
print(model2(img))

tf.Tensor([[0.9987438]], shape=(1, 1), dtype=float32)
